In [0]:
pip install google-auth-httplib2 gspread df2gspread gspread_dataframe oauth2client xlrd==1.2.0

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
ID = '13Cb-zxdQjxLcFKsjL-65xTnNf8fqBt5DT-QEY0sUe5Y'

In [0]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials as cred
import json
import gspread as gs
import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account
from googleapiclient.discovery import build
import numpy as np

In [0]:
creds = service_account.Credentials.from_service_account_file(
    '/dbfs/FileStore/shared_uploads/dssilvestre@band.com.br/python_databricks_ecb24878ff71-2.json')
scoped_credentials = creds.with_scopes(
    ['https://www.googleapis.com/auth/spreadsheets'])
service = build('sheets','v4',credentials=creds)
sheet = service.spreadsheets()
gc = gs.service_account('/dbfs/FileStore/shared_uploads/dssilvestre@band.com.br/python_databricks_ecb24878ff71-1.json')
wks = gc.open_by_key(ID)

In [0]:
ResultadosYT = spark.sql("""
with tb_videos as 
(
  select
    to_date(publishedAt) as date,
    video_id,
    video_title, 
    video_link
  from gold.youtube_dim_videos
  where lower (video_title) like '%aconteceu na semana%'
      or lower (video_title) like '%destaque da semana%'
      or lower (video_title) like '%baú %' or lower (video_title) like '%bau %'
      or lower (video_title) like '%reapresentação%' 
      or lower (video_title) like '%reapresentacao%' 
      or lower (video_title) like '%reapresentaçao%' 
      or lower (video_title) like '%reapresentacão%'
),
tb_views as 
(
  select 
    video_id, 
    coalesce(channel_id, page_id) as channel_id,
    sum(views) as views,
    sum(comments) as comentarios,
    sum(shares) as compartilhamentos,
    sum(likes) as likes,
    sum(dislikes) as dislikes,
    sum(subscribers_gained) - sum(subscribers_lost) as subscribers
  from gold.youtube_owner_basic
  group by video_id, coalesce(channel_id, page_id)
),
tb_receita as 
(
  select
    video_id,
    sum(estimated_partner_revenue) as receita
  from gold.youtube_estimated_revenue
  group by video_id
)
  select
    a.date,
    d.channel_title,
    a.video_title, 
    a.video_link,
    sum(c.receita) as receita,
    sum(b.views) as views,
    sum(b.comentarios) as comentarios,
    sum(b.compartilhamentos) as compartilhamentos,
    sum(b.likes) as likes,
    sum(b.dislikes) as dislikes,
    sum(b.subscribers) as inscritos
  from tb_videos a
  left join tb_views b on a.video_id = b.video_id
  left join tb_receita c on a.video_id = c.video_id
  left join gold.youtube_dim_channels d on d.channel_id = b.channel_id
  where a.date >= to_date('2022-01-01')
  group by 
    a.date,
    d.channel_title,
    a.video_title, 
    a.video_link
  order by a.date asc
""").toPandas()

In [0]:
set_with_dataframe(wks.worksheet("ResultadosYT"), ResultadosYT, include_index = False, include_column_header= True)